# pypmed example

In [82]:
!pip install pypmed

In [83]:
from pypmed import apis 

search_criteria = {'author_first_name': 'rachel', 
                   'author_last_name': 'gottschalk', 
                   'institution':  'university of pittsburgh at pittsburgh',
                   'publication_year': 2022
                   }

response = apis.query_pubmed_api(search_criteria=search_criteria)

a
b


In [84]:
response


[{'pmid': '35063833',
  'journal_issn': '1879-0372',
  'journal_title': 'Current opinion in immunology',
  'article_title': 'Mechanisms encoding STAT functional diversity for context-specific inflammatory responses.',
  'publication_year': '2022',
  'author_first_name': 'rachel',
  'author_last_name': 'gottschalk',
  'institution': 'university of pittsburgh at pittsburgh'}]